In [56]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import csv

csv.field_size_limit(sys.maxsize)
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words("english")

labels = ['fake', 'real']

#Snowballstemmer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

news_data = pd.read_csv('WELFake_Dataset.csv', engine='python', encoding='utf-8', on_bad_lines='skip').fillna('')

news_data_smaller = news_data.sample(n=10).reset_index(drop=True)

#uncomment for all data
news_data = news_data_smaller


def remove_punctuation(some_string):
    unpuncted_string = some_string.translate(str.maketrans("","",string.punctuation))
    return(unpuncted_string)

def apply_stemming(some_string):
    new_list = []
    for word in some_string.split():
        word = stemmer.stem(word)
        new_list.append(word)
    return(new_list)


news_data["title"] = news_data["title"].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))
news_data["title"] = news_data["title"].apply(remove_punctuation)
news_data["title"] = news_data["title"].apply(apply_stemming)

news_data["text"] = news_data["text"].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))
news_data["text"] = news_data["text"].apply(remove_punctuation)
news_data["text"] = news_data["text"].apply(apply_stemming)

print(news_data["text"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    [brussel, reuter, soccer, leagu, england, prem...
1    [buzzfeed…“trend, now”, as, liberti, blitzkrie...
2    [on, novemb, 26, 2016, obama, administr, deman...
3    [orlando, fl, the, 45th, presid, unit, state, ...
4    [washington, —, the, trump, administr, move, f...
5    [it, look, like, nation, secur, pick, monica, ...
6    [reuter, magellan, midstream, partner, lp, sai...
7    [ever, sinc, donald, trump, got, expos, audio,...
8    [havana, reuter, the, trump, administr, cuban,...
9    [14th, anniversari, his, pass, by, joachim, ha...
Name: text, dtype: object


In [57]:
print(news_data["title"][0])
print(news_data["text"][0])

['sport', 'sector', 'win', 'repriev', 'eu', 'copyright', 'reform']
['brussel', 'reuter', 'soccer', 'leagu', 'england', 'premier', 'leagu', 'germani', 'bundesliga', 'repriev', 'friday', 'eu', 'ambassador', 'agre', 'exclud', 'scope', 'copyright', 'reform', 'would', 'help', 'make', 'content', 'easili', 'avail', 'onlin', 'the', 'entertain', 'sport', 'industri', 'fierc', 'lobbi', 'european', 'commiss', 'propos', 'reform', 'eu', 'copyright', 'law', 'make', 'film', 'tv', 'program', 'avail', 'across', 'border', 'argu', 'would', 'undermin', 'financ', 'model', 'whole', 'sector', 'film', 'tv', 'program', 'often', 'financ', 'sell', 'exclus', 'distribut', 'right', 'countrybycountri', 'basi', 'right', 'show', 'sport', 'premier', 'leagu', 'soccer', 'match', 'fetch', 'billion', 'pound', 'the', 'commiss', 'said', 'seek', 'forc', 'anyon', 'make', 'content', 'avail', 'onlin', 'mere', 'make', 'easier', 'broadcast', 'obtain', 'necessari', 'right', 'eu', 'member', 'state', 'agre', 'friday', 'exclud', 'sport

In [58]:
def bow_maker(some_list):
    bow = set()
    for word in some_list:
        numbr = some_list.count(word)
        bow.add((word, numbr))
    return(bow)

print(news_data["title"].apply(bow_maker)[0])

{('sector', 1), ('win', 1), ('repriev', 1), ('sport', 1), ('eu', 1), ('copyright', 1), ('reform', 1)}


In [59]:
word_frequency = {}
wordPerLine = {}
allwordsAmount = {}
allTokens = []
counter = 0
for line in news_data["text"]:
    #print(line)
    for token in line:
        if token not in wordPerLine.keys():
            wordPerLine[token] = 1
            allwordsAmount[token] = 1
            if token not in allTokens:
              allTokens.append(token)
        else:
            wordPerLine[token] += 1
            allwordsAmount[token] += 1
    word_frequency[counter] = wordPerLine
    wordPerLine = {}
    counter = counter +1

bowResults = []
for line in news_data["text"]:
    vector=[]
    for w in allTokens:
        vector.append(line.count(w))
    print(vector)
    bowResults.append(vector) 

[1, 1, 2, 3, 1, 2, 1, 1, 1, 2, 7, 1, 2, 2, 2, 2, 3, 2, 2, 6, 6, 1, 5, 5, 5, 1, 3, 1, 1, 1, 3, 4, 3, 1, 3, 6, 5, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 5, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 